In [1]:
import math
import pandas as pd
import torch

from src.DataLoader import CaltechBirdsDataset

In [2]:
data_dict = pd.DataFrame()
dataset    = CaltechBirdsDataset(train=True)
birdloader = torch.utils.data.DataLoader(dataset, batch_size=32, shuffle=True)
for data, images in birdloader:
    data_pd = pd.DataFrame(data)
    data_dict = pd.concat((data_dict, data_pd))
   

In [3]:
data_dict

,trainset,file_name,class_id,one_hot,x,y,w,h,has_bill_shape,has_wing_color,...,has_size,has_shape,has_back_pattern,has_tail_pattern,has_belly_pattern,has_primary_color,has_leg_color,has_bill_color,has_crown_color,has_wing_pattern
0,1,049.Boat_tailed_Grackle/Boat_Tailed_Grackle_00...,49,48,62,59,335,210,7,1,...,2,14,4,0,4,1,1,1,1,4
1,1,082.Ringed_Kingfisher/Ringed_Kingfisher_0021_7...,82,81,207,95,141,203,6,6,...,2,1,1,1,3,6,2,12,12,0
2,1,014.Indigo_Bunting/Indigo_Bunting_0005_13832.jpg,14,13,90,57,187,315,7,1,...,5,9,1,0,0,1,6,6,1,0
3,1,014.Indigo_Bunting/Indigo_Bunting_0041_13987.jpg,14,13,218,273,189,101,8,9,...,2,14,1,3,1,1,6,6,1,1
4,1,020.Yellow_breasted_Chat/Yellow_Breasted_Chat_...,20,19,205,126,120,139,7,12,...,2,14,1,1,1,6,12,12,12,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5,1,038.Great_Crested_Flycatcher/Great_Crested_Fly...,38,37,272,138,132,130,7,2,...,2,7,3,4,1,2,2,6,2,3
6,1,110.Geococcyx/Geococcyx_0075_104334.jpg,110,109,19,21,421,362,2,12,...,2,7,3,3,1,6,6,6,12,3
7,1,171.Myrtle_Warbler/Myrtle_Warbler_0107_166917.jpg,171,170,67,69,181,401,7,0,...,2,14,0,0,3,8,12,8,8,2
8,1,200.Common_Yellowthroat/Common_Yellowthroat_00...,200,199,177,18,201,263,7,2,...,5,14,4,4,4,2,11,2,7,4


In [4]:
def entropy(data):
    H = [-data[i] * math.log(data[i]) for i in range(1,201) if data[i] > 0]
    return(sum(H))

probabilities = pd.DataFrame()
for concept_name in dataset.concept_names:
    concept_pivot = data_dict.pivot_table(index=concept_name, columns='class_id', aggfunc='size', fill_value=0)
    concept_probs = concept_pivot.div(concept_pivot.sum(axis=1), axis=0)
    concept_probs.insert(0, 'concept_name', concept_name)
    concept_entropy = concept_probs.apply(func=entropy, axis=1,raw=True, result_type='reduce' )
    #concept_probs['entropy'] = concept_entropy
    concept_probs.insert(1, 'entropy', concept_entropy)
    probabilities = pd.concat((probabilities, concept_probs))
    
    

In [5]:
probabilities

class_id,concept_name,entropy,1,2,3,4,5,6,7,8,...,191,192,193,194,195,196,197,198,199,200
0,has_bill_shape,4.507321,0.012048,0.012048,0.000000,0.018072,0.012048,0.018072,0.000000,0.018072,...,0.000000,0.006024,0.000000,0.006024,0.006024,0.000000,0.000000,0.000000,0.000000,0.006024
1,has_bill_shape,3.659936,0.000000,0.007143,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.057143,0.078571,0.078571,0.007143,0.028571,0.021429,0.000000,0.000000
2,has_bill_shape,4.370537,0.001242,0.001242,0.002484,0.001242,0.000000,0.000000,0.000000,0.002484,...,0.024845,0.018634,0.004969,0.012422,0.001242,0.004969,0.002484,0.018634,0.000000,0.002484
3,has_bill_shape,3.513724,0.010753,0.032258,0.010753,0.021505,0.000000,0.000000,0.010753,0.032258,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,has_bill_shape,2.651041,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.007143,0.007143,0.000000,0.000000,0.007143,0.000000,0.007143,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,has_wing_pattern,4.991896,0.012232,0.015291,0.009174,0.007645,0.012232,0.012232,0.010703,0.018349,...,0.000000,0.000000,0.000000,0.001529,0.000000,0.001529,0.001529,0.000000,0.000000,0.004587
1,has_wing_pattern,4.931033,0.006459,0.009395,0.010570,0.012331,0.012331,0.001762,0.008221,0.007046,...,0.001174,0.000000,0.011744,0.000000,0.005872,0.001762,0.001174,0.001762,0.002349,0.010570
2,has_wing_pattern,4.259692,0.000000,0.002174,0.000000,0.002174,0.000000,0.004348,0.000000,0.000000,...,0.000000,0.047826,0.002174,0.047826,0.008696,0.015217,0.015217,0.034783,0.015217,0.000000
3,has_wing_pattern,4.985982,0.004781,0.002049,0.000000,0.000683,0.000000,0.008880,0.003415,0.002732,...,0.000683,0.004098,0.005464,0.002732,0.009563,0.010929,0.007514,0.005464,0.008880,0.002049


In [6]:
test_dict = pd.DataFrame()
testset    = CaltechBirdsDataset(train=False)
birdloader = torch.utils.data.DataLoader(testset, batch_size=32, shuffle=True)
for data, images in birdloader:
    data_pd = pd.DataFrame(data)
    test_dict = pd.concat((test_dict, data_pd))

In [7]:
test_dict

,trainset,file_name,class_id,one_hot,x,y,w,h,has_bill_shape,has_wing_color,...,has_size,has_shape,has_back_pattern,has_tail_pattern,has_belly_pattern,has_primary_color,has_leg_color,has_bill_color,has_crown_color,has_wing_pattern
0,0,061.Heermann_Gull/Heermann_Gull_0118_45626.jpg,61,60,151,69,271,202,6,6,...,4,4,3,1,1,6,12,12,13,3
1,0,156.White_eyed_Vireo/White_Eyed_Vireo_0051_159...,156,155,80,62,165,377,7,7,...,2,14,1,4,1,6,6,12,6,4
2,0,115.Brewer_Sparrow/Brewer_Sparrow_0005_107443.jpg,115,114,164,108,154,325,8,2,...,5,14,4,4,4,2,2,2,2,4
3,0,057.Rose_breasted_Grosbeak/Rose_Breasted_Grosb...,57,56,44,23,334,282,8,12,...,2,14,1,1,4,12,12,15,12,4
4,0,089.Hooded_Merganser/Hooded_Merganser_0007_791...,89,88,139,102,256,126,7,2,...,4,1,1,3,4,2,2,12,12,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29,0,095.Baltimore_Oriole/Baltimore_Oriole_0056_883...,95,94,92,95,255,360,7,7,...,2,14,0,1,1,7,6,15,12,4
30,0,041.Scissor_tailed_Flycatcher/Scissor_Tailed_F...,41,40,205,50,154,320,8,0,...,2,13,0,1,4,11,12,12,13,0
31,0,069.Rufous_Hummingbird/Rufous_Hummingbird_0074...,69,68,0,30,428,433,4,0,...,5,7,0,0,0,2,0,12,0,0
0,0,144.Common_Tern/Common_Tern_0105_149841.jpg,144,143,136,113,237,104,1,0,...,3,11,4,0,1,13,0,2,12,0


In [47]:
concept_list = ['has_eye_color', 'has_bill_length','has_belly_pattern', 'has_breast_pattern']#,   'has_bill_shape', 'has_breast_pattern', 'has_crown_color', 
               # 'has_forehead_color', 'has_shape', 'has_size', 'has_throat_color']
breaker = 2
count, correct, ones = 0, 0, 0
for index, record in test_dict.iterrows():
    class_concept_probabilities = pd.DataFrame()
    for concept_name in dataset.concept_names:
        concept_value = record[concept_name]
        tests = (probabilities['concept_name']==concept_name,   probabilities.index==concept_value)
        class_concept_probabilities = pd.concat((class_concept_probabilities, probabilities[tests[0] & tests[1]]))      
    class_probabilities = class_concept_probabilities[[class_id for class_id in range(1,201)]].prod(axis=0, numeric_only=True)
    count += 1
    correct += (record['class_id'] == class_probabilities.idxmax())
    ones    += (1                  == class_probabilities.idxmax())
    if breaker == count: break
    

In [48]:
record

trainset                                                                0
file_name               156.White_eyed_Vireo/White_Eyed_Vireo_0051_159...
class_id                                                              156
one_hot                                                               155
x                                                                      80
y                                                                      62
w                                                                     165
h                                                                     377
has_bill_shape                                                          7
has_wing_color                                                          7
has_upperparts_color                                                    6
has_underparts_color                                                    7
has_breast_pattern                                                      1
has_back_color                        

In [57]:
class_concept_probabilities

class_id,concept_name,entropy,1,2,3,4,5,6,7,8,...,191,192,193,194,195,196,197,198,199,200
7,has_bill_shape,4.987529,0.000000,0.000000,0.000877,0.000438,0.000000,0.000877,0.000000,0.003507,...,0.003946,0.006138,0.007453,0.003069,0.007453,0.009645,0.008768,0.005261,0.012275,0.010083
7,has_wing_color,3.639993,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.014815
6,has_upperparts_color,4.952761,0.003613,0.002168,0.007948,0.000000,0.003613,0.008671,0.006503,0.004335,...,0.000723,0.000723,0.000723,0.004335,0.000000,0.000723,0.001445,0.005780,0.000723,0.005780
7,has_underparts_color,3.870645,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.030914
1,has_breast_pattern,5.147261,0.004835,0.006649,0.006044,0.006649,0.008160,0.001209,0.002418,0.004835,...,0.006346,0.007555,0.008160,0.000604,0.007555,0.006649,0.006346,0.004835,0.003324,0.007555
6,has_back_color,4.926713,0.004125,0.000825,0.008251,0.000000,0.004125,0.005776,0.004125,0.004125,...,0.000000,0.000825,0.000825,0.002475,0.000000,0.002475,0.001650,0.005776,0.000825,0.007426
3,has_tail_shape,5.162101,0.003348,0.001116,0.001116,0.002790,0.001674,0.005022,0.001674,0.001674,...,0.004464,0.003348,0.004464,0.006138,0.003906,0.004464,0.005580,0.004464,0.003906,0.005022
6,has_upper_tail_color,4.932287,0.004141,0.002070,0.012422,0.001035,0.004141,0.004141,0.005176,0.004141,...,0.000000,0.000000,0.000000,0.003106,0.000000,0.002070,0.001035,0.008282,0.000000,0.006211
7,has_head_pattern,4.963520,0.002404,0.000000,0.008413,0.001202,0.000000,0.008413,0.002404,0.008413,...,0.004808,0.002404,0.001202,0.000000,0.001202,0.010817,0.008413,0.003606,0.002404,0.003606
7,has_breast_color,3.825369,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.039764


In [51]:
round(class_probabilities / sum(class_probabilities),2)

class_id
1      0.0
2      0.0
3      0.0
4      0.0
5      0.0
      ... 
196    0.0
197    0.0
198    0.0
199    0.0
200    0.0
Length: 200, dtype: float64

In [28]:
ones / count

0.00017259233690024162